In [127]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import datetime
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

connect to the wildfire sqlite database and extract a df from it

In [162]:
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT * from Fires", conn)[::100]

Turn Date and CONT_DATE to datetime object.

In [163]:
def to_date(x):
    date = datetime.datetime(x['FIRE_YEAR'], 1, 1) + datetime.timedelta(x['DISCOVERY_DOY'] - 1)
    return date

In [164]:
df['DATE']=df.apply(to_date, axis=1)
df['CONT_DATE'] = df.apply(to_date, axis=1)

create new features denoting the month and day in month the fire was discovered in.

In [165]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['DAY'] = df['DATE'].dt.day
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year

create a feature that checks if the fire was discovered around the date of 4th of july

In [132]:
def july4(x):
    if x['MONTH']==7:
        if 2<=x['DAY'] and x['DAY']<=6:
            return 1
    return 0

In [133]:
df['JULY4']=df.apply(july4, axis=1)

Adds a features that denote the amount of time passing between fire discovery and fire contaiment

In [134]:
def to_cont_date(x):
    if x['CONT_DOY']>0:
        date = datetime.datetime(x['FIRE_YEAR'], 1, 1) + datetime.timedelta(x['CONT_DOY'] - 1)
        if (date-x['DATE']).days<0:
            date = datetime.datetime(x['FIRE_YEAR']+1, 1, 1) + datetime.timedelta(x['CONT_DOY'] - 1)
        return date

    return datetime.datetime(1990, 1, 1)

In [135]:
def is_cont_date(x):
    if x['C_DATE']==datetime.datetime(1990, 1, 1):
        return 0
    return 1

In [136]:
def exists_times(x):
    if not x['DISCOVERY_TIME'] or not x['CONT_TIME']:
        return 0
    return 1

In [137]:
def add_hour(x):
    if x['DISCOVERY_TIME']:
        if int(x['DISCOVERY_TIME'][2:])>30:
            return int(x['DISCOVERY_TIME'][:2])+1
        return int(x['DISCOVERY_TIME'][:2])
    return -1

In [138]:
def add_hour_do_dis_date(x):
    total = x['DATE']
    if x['HOUR']>=0:
        total += datetime.timedelta(hours = int(x['DISCOVERY_TIME'][:2]), minutes=int(x['DISCOVERY_TIME'][2:]))
    return total

In [139]:
def add_cont_hour(x):
    if x['CONT_TIME']:
        if int(x['CONT_TIME'][2:])>30:
            return int(x['CONT_TIME'][:2])+1
        return int(x['CONT_TIME'][:2])
    return -1

In [140]:
def add_hour_to_cont_date(x):
    if x['C_DATE']==datetime.datetime(1990, 1, 1):
        return x['C_DATE']
    total = x['C_DATE']
    if x['C_HOUR']>=0:
        total += datetime.timedelta(hours = int(x['CONT_TIME'][:2]), minutes=int(x['CONT_TIME'][2:]))
    return total 

In [141]:
def duration(x):
    if x['C_DATE']==datetime.datetime(1990, 1, 1):
        return -1
    else:
        if x['EXISTS_TIMES']==1:
            difference = x['C_DATE']-x['DATE']
        else:
            difference = (x['C_DATE']+ datetime.timedelta(hours = 23, minutes=59))-x['DATE']  
        return difference

In [142]:
def duration_days(x):
    if x['DURATION']==-1:
        return -1
    return x['DURATION'].days

In [143]:
def duration_hours(x):
    if x['DURATION']==-1:
        return -1
    return x['DURATION'].days*24+x['DURATION'].seconds//3600

In [144]:
def duration_30m(x):
    if x['DURATION']==-1:
        return 0
    if x['DURATION']== datetime.timedelta(hours = 0, minutes=30):
        return 1
    return 0

In [145]:
def duration_1h(x):
    if x['DURATION']==-1:
        return 0
    if x['DURATION']== datetime.timedelta(hours = 1, minutes=0):
        return 1
    return 0

In [146]:
def duration_same_d(x):
    if x['DURATION_DAYS']==-1:
        return 0
    if x['DURATION_DAYS']==0:
        return 1
    return 0

In [147]:
df['C_DATE'] = df.apply(to_cont_date, axis=1)
df['C_DATE_EXISTS'] = df.apply(is_cont_date, axis=1)
df['C_DATE'] = pd.to_datetime(df['C_DATE'])
df['EXISTS_TIMES']=df.apply(exists_times, axis=1)
df['HOUR']=df.apply(add_hour, axis=1)
df['DATE']=df.apply(add_hour_do_dis_date, axis=1)
df['C_HOUR']=df.apply(add_cont_hour, axis=1)
df['C_DATE']=df.apply(add_hour_to_cont_date, axis=1)
df['DURATION']=df.apply(duration, axis=1)
df['DURATION_DAYS']=df.apply(duration_days, axis=1)
df['DURATION_HOURS']=df.apply(duration_hours, axis=1)
df['DUR_30M']=df.apply(duration_30m, axis=1)
df['DUR_1H']=df.apply(duration_1h, axis=1)
df['DUR_1D']=df.apply(duration_same_d, axis=1)

Drop all features that may cause leakage or are indexes in databases

In [166]:
df_dropped = df.drop(columns=["FPA_ID", "SOURCE_SYSTEM_TYPE", "SOURCE_SYSTEM", "NWCG_REPORTING_AGENCY",
                             "NWCG_REPORTING_UNIT_ID", "NWCG_REPORTING_UNIT_NAME", "SOURCE_REPORTING_UNIT",
                             "SOURCE_REPORTING_UNIT_NAME", "Shape", "OBJECTID", "LOCAL_FIRE_REPORT_ID",
                             "LOCAL_INCIDENT_ID", "FIRE_CODE", "FIRE_NAME", "ICS_209_INCIDENT_NUMBER",
                             "ICS_209_NAME", "MTBS_ID", "MTBS_FIRE_NAME", "COMPLEX_NAME", "FIPS_CODE", "FIPS_NAME",
                             "OWNER_CODE", "DATE", "FIRE_YEAR", "DISCOVERY_DATE", "DISCOVERY_TIME", 
                             "CONT_DATE", "CONT_DOY", "CONT_TIME", "C_DATE", "DURATION"])

KeyError: "['C_DATE' 'DURATION'] not found in axis"

Turn the dataframe into a GeoDataFrame to work with geodata

In [149]:
df_dropped = gpd.GeoDataFrame(df_dropped, geometry=gpd.points_from_xy(df_dropped.LONGITUDE, df_dropped.LATITUDE), crs="EPSG:4326")

A function that gets a path to a csv containing a geometry column and turns it into a geodataframe

In [150]:
def get_gdf(file_path):
        gdf = pd.read_csv(file_path)
        return gpd.GeoDataFrame(gdf.loc[:, [c for c in gdf.columns if c != "geometry"]], 
                                geometry=gpd.GeoSeries.from_wkt(gdf["geometry"]))

get distance of fire from railroad

In [151]:
def get_distance_to_rails_feature(df):
    gdf = get_gdf("datasets/rail_north_america/rails_geo.csv")

    gdf.crs = "EPSG:4326"

    gdf = gdf.to_crs("EPSG:3857")
    df.crs = gdf.crs
    res = gpd.sjoin_nearest(df, gdf, distance_col="distance_to_rails", how="left")
    res = res.drop(["index_right", "scalerank", "featurecla", "sov_a3", "uident", "add", "natrlscale", "continent"], axis=1)
    return res

getting population density per location of fire feature

In [152]:
def get_pop(df):
    gdf = get_gdf("datasets/population_density/data_populations_usa.csv")
    gdf.crs = "EPSG:4326"

    gdf = gdf.to_crs("EPSG:3857")
    
    res = gpd.sjoin_nearest(df, gdf, how="left")
    res = res.drop(["index_right"], axis=1)
    
    return res

A general function that gets a dataframe a file_path to a dataframe with geometry column and adds to the df the columns of the nearest point in the geodataframe with a feature for distance from that point

In [153]:
def get_distance_feature(df, file_path, feature_name):
    gdf = get_gdf(file_path)

    gdf.crs = "EPSG:4326"

    gdf = gdf.to_crs("EPSG:3857")
    df.crs = gdf.crs

    res = gpd.sjoin_nearest(df, gdf, distance_col=feature_name, how="left")
    res = res.drop(["OBJECTID", "index_right"], axis=1)
    return res

get the distance from nearest city to the fire and the name of that city

In [154]:
def get_city_and_distance_feature(df, file_path, feature_name):
    gdf = get_gdf(file_path)

    gdf.crs = "EPSG:4326"

    gdf = gdf.to_crs("EPSG:3857")
    df.crs = gdf.crs

    res = gpd.sjoin_nearest(df, gdf, distance_col=feature_name, how="left")
    res = res.drop(["index_right"], axis=1)
    return res

adds the average, min, max, prcp and pan evaporation features in the area of the fire before the fire started

In [155]:
def date_to_check(row):
    day_to_check = row['DISCOVERY_DOY'] - (1 + (row['DISCOVERY_DOY'] - 1) % 3)
    date = datetime.datetime(row['YEAR'], 1, 1) + datetime.timedelta(int(day_to_check))
    
    if date.year < 1992:
        return 1992, 1, 1
    else:
        return date.year, date.month, date.day

def temps_func(df):
    gdf = pd.read_csv("datasets/temps_dfs/temps_area_codes.csv")
    gdf = gpd.GeoDataFrame(gdf.loc[:, [c for c in gdf.columns if c != "geometry"]],
                           geometry=gpd.GeoSeries.from_wkt(gdf["geometry"]))
    
    gdf.crs = "EPSG:4326"

    gdf = gdf.to_crs("EPSG:3857")
    df.crs = gdf.crs

    df = gpd.sjoin_nearest(df, gdf, how="left")
    
    df = df.drop(["index_right"], axis=1)

    df["day_remove"] = df["DAY"]
    df["month_remove"] = df["MONTH"]
    df["year_remove"] = df["YEAR"]

    dates = np.array(list(df.apply(date_to_check, axis=1)))

    df["DAY"] = dates[:, 2]
    df["MONTH"] = dates[:, 1]
    df["YEAR"] = dates[:, 0]

    gdf = pd.read_csv(f"datasets/temps_dfs/temps_area_code_dates.csv")

    gdf["YEAR"] = gdf["YEAR"].astype("int32")
    df["YEAR"] = df["YEAR"].astype("int32")
    gdf["MONTH"] = gdf["MONTH"].astype("int32")
    df["MONTH"] = df["MONTH"].astype("int32")
    gdf["DAY"] = gdf["DAY"].astype("int32")
    df["DAY"] = df["DAY"].astype("int32")

    df = pd.merge(df, gdf, on=["DAY", "MONTH", "YEAR", "area_code"], how="left")

    df["DAY"] = df["day_remove"]
    df["MONTH"] = df["month_remove"]
    df["YEAR"] = df["year_remove"]
    df = df.drop(["area_code", "day_remove", "month_remove", "year_remove"], axis=1)
    df = df.drop_duplicates(subset=["FOD_ID"])
    
    return df

adds demographic features and smoking stats per state

In [156]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "Virgin Islands": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
    "United States": "United States"
}

def map_states(y):
    return us_state_to_abbrev.get(y, "____")

# adds data from https://www.kff.org/statedata/
def get_info_by_states(df, address):
    df = df.sort_values(by=["YEAR"])
    df["YEAR"] = pd.to_numeric(df["YEAR"])

    csv_files = ['citenzship',
                 'race',
                 'tobacco',
                 'demographics_with_years']
    
    for table in csv_files:
        curr = pd.read_csv(f"{address}/{table}.csv")
        curr = curr.dropna()
        if table == 'tobacco':
            curr.rename(columns = {'State':'STATE', "Year":"YEAR"}, inplace = True)
        else:
            curr.rename(columns = {'Location':'STATE'}, inplace = True)
            
        curr['STATE'] = curr['STATE'].apply(map_states)
        
        if (table == 'tobacco') or (table == "demographics_with_years"):
            curr["YEAR"] = pd.to_numeric(curr["YEAR"])
            curr = curr.sort_values(by=["YEAR"])
            
            df = pd.merge_asof(df, curr, on="YEAR", by='STATE', direction="nearest")
            
        else:
            df = pd.merge(df, curr, on=['STATE'], how="left")
    
    df = df.drop_duplicates(subset=["FOD_ID"])
    df = df.applymap(lambda x: x.strip('%') if isinstance(x, str) else x)
    return df

turns the fire class feature into an ordinal feature

In [157]:
def turn_fire_class_to_ordinal(row):
    if row["FIRE_SIZE_CLASS"] == "A":
        return 0
    elif row["FIRE_SIZE_CLASS"] == "B":
        return 1
    elif row["FIRE_SIZE_CLASS"] == "C":
        return 2
    elif row["FIRE_SIZE_CLASS"] == "D":
        return 3
    elif row["FIRE_SIZE_CLASS"] == "E":
        return 4
    elif row["FIRE_SIZE_CLASS"] == "F":
        return 5
    elif row["FIRE_SIZE_CLASS"] == "G":
        return 6
    else:
        return -1

a function that checks if PCA based features help the model diffrintiate between sampeles, in our testing it seems tha pca features caused overfit and as such we do not use them.

In [158]:
def check_pca_features(df):
    features = [i for i in df.columns if i not in ['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']]

    for i in range(2, 21):
        df_with_embd = df.copy()
        for j in range(2):

            if j == 0:
                pca = PCA(n_components=i)
            else:
                pca = PCA(n_components=i, whiten=True)

            embd = pca.fit_transform(df[features])

            for k in range(i):
                df_with_embd[f"embd{k}"] = embd[:, k]

            X = df_with_embd.drop(columns=['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR'])
            y = df_with_embd['STAT_CAUSE_DESCR']
            X_train, X_test, y_train, y_test = train_test_split(X, y)

            model = RandomForestClassifier(n_jobs=-1)
            model.fit(X_train, y_train)

            preds = model.predict(X_test)

            cr_dict = classification_report(y_test, preds, output_dict=True)
            accuracy = cr_dict["accuracy"]
            macro_f1 = cr_dict["macro avg"]["f1-score"]

            if j == 0:
                print(f"with {i} components accuracy: {accuracy} and macro f1: {macro_f1}")
            else:
                print(f"with {i} and whiten components accuracy: {accuracy} and macro f1: {macro_f1}")

a function that gets a dataframe and adds features to it

In [159]:
def get_features(df):
    df = df.to_crs("EPSG:3857")
    print("getting pop feature...", end="")
    df = get_pop(df)
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    
    print("getting rails feature...", end="")
    df = get_distance_to_rails_feature(df)
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    
    print("getting powerline feature...", end="")
    df = get_distance_feature(df, "datasets/powerlines/powerlines.csv", "distance_to_powerline")
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    
    print("getting landfill feature...", end="")
    df = get_distance_feature(df, "datasets/landfill_locations/Landfill_Locations.csv", "distance_to_landfill")
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    
    print("getting home_parks_distance feature...", end="")
    df = get_distance_feature(df, "datasets/mobile_home_parks/Mobile_Home_Parks.csv", "home_parks_distance")
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    
    print("getting public school distance feature...", end="")
    df = get_distance_feature(df, "datasets/schools/Public_Schools.csv", "public_school_distance")
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    
    print("getting city and distance feature...", end="")
    df = get_city_and_distance_feature(df, "datasets/cities/CityBoundaries.csv", "city_distance")
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    
    print("getting temperature features...", end="")
    df = temps_func(df)
    print("Done")
    
    print("getting ordinal fire class...", end="")
    df["fire_class_ordinal"] = df.apply(turn_fire_class_to_ordinal, axis=1)
    print("Done")
    
    print("getting usa by state features...", end="")
    df = get_info_by_states(df, "datasets/usa_by_state")
    print("Done")
    
    df["max_temp"].fillna(value=df["max_temp"].mean(), inplace=True)
    df["min_temp"].fillna(value=df["min_temp"].mean(), inplace=True)
    df["prcp"].fillna(value=df["prcp"].mean(), inplace=True)
    df["Pan_evaporation"].fillna(value=df["Pan_evaporation"].mean(), inplace=True)
    df["avg_temp"].fillna(value=df["avg_temp"].mean(), inplace=True)
    
    df = df.fillna(0)
    
    return df

A function that gets a geodataframe and adds the city name feature to ready it for the one hot encoder

In [160]:
def get_df_ready_for_encoder(df):
    print("getting city and distance feature...", end="")
    df = get_city_and_distance_feature(df, "datasets/cities/CityBoundaries.csv", "city_distance")
    df = df.groupby("FOD_ID").first().reset_index()
    print("Done")
    return df

In [161]:
df_dropped = get_df_ready_for_encoder(df_dropped)

getting city and distance feature...

ProjError: x, y, z, and time must be same size

one hot encode the STATE OWNER_DESCR and NAME features

In [ ]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
encoder.fit(df_dropped[["STATE", "OWNER_DESCR", "NAME"]])
COLUMNS_NAME_ONE_HOT = list(encoder.categories_[0]) + list(encoder.categories_[1]) + list(encoder.categories_[2])

The general preprocessing function that receives a dataframe from the wildifre dataset and preprocesses it for the use of our model

In [ ]:
def preprocess(df, encoder):
    print("working on dates...", end="")
    df['DATE']=df.apply(to_date, axis=1)
    df['CONT_DATE'] = df.apply(to_date, axis=1)
    
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['DAY'] = df['DATE'].dt.day
    df['MONTH'] = df['DATE'].dt.month
    df['YEAR'] = df['DATE'].dt.year
    
    df['JULY4']=df.apply(july4, axis=1)
    
    df['C_DATE'] = df.apply(to_cont_date, axis=1)
    df['C_DATE_EXISTS'] = df.apply(is_cont_date, axis=1)
    df['C_DATE'] = pd.to_datetime(df['C_DATE'])
    df['EXISTS_TIMES']=df.apply(exists_times, axis=1)
    df['HOUR']=df.apply(add_hour, axis=1)
    df['DATE']=df.apply(add_hour_do_dis_date, axis=1)
    df['C_HOUR']=df.apply(add_cont_hour, axis=1)
    df['C_DATE']=df.apply(add_hour_to_cont_date, axis=1)
    df['DURATION']=df.apply(duration, axis=1)
    df['DURATION_DAYS']=df.apply(duration_days, axis=1)
    df['DURATION_HOURS']=df.apply(duration_hours, axis=1)
    df['DUR_30M']=df.apply(duration_30m, axis=1)
    df['DUR_1H']=df.apply(duration_1h, axis=1)
    df['DUR_1D']=df.apply(duration_same_d, axis=1)
    print("Done")
    
    df = df.drop(columns=["FPA_ID", "SOURCE_SYSTEM_TYPE", "SOURCE_SYSTEM", "NWCG_REPORTING_AGENCY",
                         "NWCG_REPORTING_UNIT_ID", "NWCG_REPORTING_UNIT_NAME", "SOURCE_REPORTING_UNIT",
                         "SOURCE_REPORTING_UNIT_NAME", "Shape", "OBJECTID", "LOCAL_FIRE_REPORT_ID",
                         "LOCAL_INCIDENT_ID", "FIRE_CODE", "FIRE_NAME", "ICS_209_INCIDENT_NUMBER",
                         "ICS_209_NAME", "MTBS_ID", "MTBS_FIRE_NAME", "COMPLEX_NAME", "FIPS_CODE", "FIPS_NAME",
                         "OWNER_CODE", "DATE", "FIRE_YEAR", "DISCOVERY_DATE", "DISCOVERY_TIME", 
                         "CONT_DATE", "CONT_DOY", "CONT_TIME", "C_DATE", "DURATION"])
    
    df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE), crs="EPSG:4326")
    
    df = get_features(df)
    
    df = df.rename(columns={"z": "population density"})
    df = df.drop(columns=["FOD_ID", "geometry"])
    
    print("One hot encoding...", end="")
    one_hots = pd.DataFrame(encoder.transform(df[["STATE", "OWNER_DESCR", "NAME"]]),
                            index=df.index, columns=COLUMNS_NAME_ONE_HOT)
    print("Done")

    df = pd.concat([df, one_hots], axis=1)

    df = df.drop(columns=["COUNTY", "STATE", "FIRE_SIZE_CLASS", "OWNER_DESCR", "NAME", "Unnamed: 0"])
    
    features = [i for i in df.columns if i not in ['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']]
    df[features] = df[features].astype("float")
    
    print("Interaction of smokers and population size...", end="")
    df["Smoke everyday number"] = df["Smoke everyday"] * df["Total"]
    df["Smoke some days number"] = df["Smoke some days"] * df["Total"]
    df["Former smoker number"] = df["Former smoker"] * df["Total"]
    df["Never smoked number"] = df["Never smoked"] * df["Total"]
    print("Done")
    
    features = [i for i in df.columns if i not in ['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']]
    
    print("Normalizing data...", end="")
    scaler = StandardScaler()
    df_normalized = pd.DataFrame(scaler.fit_transform(df[features]), columns=[features])
    df[features] = df_normalized
    print("Done")
    
    return df